# Even Fibonacci numbers

>Each new term in the Fibonacci sequence is generated by adding the previous two terms. By starting with 1 and 2, the first 10 terms will be:
>
>1, 2, 3, 5, 8, 13, 21, 34, 55, 89, ...
>
>By considering the terms in the Fibonacci sequence whose values do not exceed four million, find the sum of the even-valued terms.



The terms of the Fibonacci sequence get large pretty fast, so there won’t be very many of them that are smaller than 4,000,000:

In [1]:
a, b = 1, 2
println(a)
while b < 4000000
    println(b)
    a, b = b, a + b
end

1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025
121393
196418
317811
514229
832040
1346269
2178309
3524578


That’s 32 terms, and only every third term is even (odd + even = odd; even + odd = odd; odd + odd = even; …) so that’s fewer terms still. We can really just collect the terms and then add them up:

In [2]:
function evenfibs(max)
    a, b = zero(BigInt), one(BigInt)
    terms = []
    while b < max
        iseven(b) && push!(terms, b)
        a, b = b, a + b
    end
    terms
end

sum(evenfibs(4000000))

4613732

## Iterators

If we were working with a longer set of terms, it might help to write a custom iterator instead of trying to generate them all at once.

In [3]:
struct Fib
  max::Int
end

function Base.iterate(F::Fib, state=(1, 2))
  a, b = state
  if a > F.max
      return nothing
  end
  (a, (b, a + b))
end

Base.eltype(::Type{Fib}) = Int
Base.IteratorSize(F::Fib) = Base.SizeUnknown()

Now we can filter the numbers generated by the iterator and add them up:

In [4]:
sum(Iterators.filter(iseven, Fib(4000000)))

4613732

## Solution using a Julia Channel

With this model, we have a *channel* (`F`), which the `fibonacci` function writes values to and which the other code pulls values from.

In [5]:
function fibonacci(c::Channel)
  a, b = zero(BigInt), one(BigInt)
  put!(c, a)
  put!(c, b)

  while true
    put!(c, a + b)
    a, b = b, a + b
  end
end

total = 0
F = Channel(fibonacci)
while (n = take!(F)) < 4000000
  iseven(n) && (total += n)
end
total

4613732

The terms in this sequence grow exponentially and so we only need to consider a few of them before they equal or exceed $4\times 10^{6}$. Indeed, only something like 34 terms need to be examined. This means that a direct approach of calculating them and adding them up one at a time will work quickly.

A slight improvement over calculating each term in succession is to note that only every third term (terms number $2,5,8,\ldots$) are even. It isn't hard to work out how to advance through the sequence by $3$ terms at a time.

If we set $$X_n=\left(\begin{array}{c} f_n\\f_{n-1}\end{array}\right)$$ then 

$$X_{n+1}=\left(\begin{array}{cc} 1 &1 \\ 1&0 \end{array}\right)\cdot X_{n-1}$$ so that the vectors 

$$X_{2+3k}=\left(\left(\begin{array}{cc} 1 &1 \\ 1&0 \end{array}\right)^3\right)^k\cdot X_{2}$$ are the ones whose first component is an even Fibonacci number.
